In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

home_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')


y = home_data['SalePrice']
home_data.drop(['SalePrice'], axis=1, inplace=True)



home_data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [15]:
missing_val_count_by_column = (home_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
SetNullToZero = ['LotFrontage','MasVnrArea']
# Columns with 'Not Applicable'  in NaN
SetNullToNo = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
            'BsmtFinType2','Electrical','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond',
            'PoolQC','Fence','MiscFeature']
#Columns with median in NaN
SetNullToMed = ['GarageYrBlt']

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [16]:
home_data[SetNullToZero] = home_data[SetNullToZero].fillna(value = 0)
home_data[SetNullToNo] = home_data[SetNullToNo].fillna(value = 'Not Applicable')
home_data[SetNullToMed] = home_data[SetNullToMed].fillna(home_data['GarageYrBlt'].median())
home_data.head(15)

test_data[SetNullToZero] = test_data[SetNullToZero].fillna(value = 0)
test_data[SetNullToNo] = test_data[SetNullToNo].fillna(value = 'Not Applicable')
test_data[SetNullToMed] = test_data[SetNullToMed].fillna(test_data['GarageYrBlt'].median())

X_train, X_valid, y_train, y_valid = train_test_split(home_data, y,
                                                    train_size=0.8, test_size=0.2,random_state=0)

X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(test_data)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [17]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor


model = XGBRegressor(random_state = 0,n_estimators = 1000,learning_rate = 0.05,n_jobs = 4)

model.fit(X_train,y_train,early_stopping_rounds = 5,eval_set=[(X_valid,y_valid)],verbose = False) # Your code here

# Get predictions
predictions = model.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions,y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Mean Absolute Error: 16511.47514447774


In [18]:
final_predictions = model.predict(X_test)

In [19]:
output = pd.DataFrame({'Id': test_data.Id,
                        'SalePrice': final_predictions})
output.to_csv('submission.csv', index=False)